In [2]:
import pandas as pd

In [ ]:
# 파일 읽기
try:
    df_rate_raw = pd.read_csv('시장금리(1995.5월부터).csv', encoding='cp949')
    df_kospi_raw = pd.read_csv('코스피지수(1981.5월부터).csv', encoding='cp949')
    df_exch_raw = pd.read_csv('USD_KRW(1981.5월부터).csv', encoding='cp949')
except:
    df_rate_raw = pd.read_csv('시장금리(1995.5월부터).csv', encoding='utf-8-sig')
    df_kospi_raw = pd.read_csv('코스피지수(1981.5월부터).csv', encoding='utf-8-sig')
    df_exch_raw = pd.read_csv('USD_KRW(1981.5월부터).csv', encoding='utf-8-sig')

In [46]:
# 신용스프레드 전처리
dates = df_rate_raw.columns[4:]
treasury = df_rate_raw.iloc[0, 4:].values
corporate = df_rate_raw.iloc[1, 4:].values

df_rate = pd.DataFrame({'date': dates, 'treasury': treasury, 'corporate': corporate})
df_rate['date'] = pd.to_datetime(df_rate['date'], format='%Y/%m')
df_rate = df_rate.set_index('date')

df_rate['treasury'] = pd.to_numeric(df_rate['treasury'], errors='coerce')
df_rate['corporate'] = pd.to_numeric(df_rate['corporate'], errors='coerce')
df_rate['credit_spread'] = df_rate['corporate'] - df_rate['treasury']

# 2004년~2026년 필터링 후 저장
df_spread_final = df_rate[['credit_spread']].loc['2004-01-01':].sort_index()
df_spread_final.to_csv('전처리/전처리_신용스프레드.csv', encoding='utf-8-sig')


In [47]:
# 코스피 전처리
df_kospi = df_kospi_raw[['날짜', '종가']].copy()
df_kospi.columns = ['date', 'kospi']
df_kospi['date'] = pd.to_datetime(df_kospi['date'].astype(str).str.replace(' ', ''))
df_kospi = df_kospi.set_index('date')
df_kospi['kospi'] = df_kospi['kospi'].astype(str).str.replace(',', '').astype(float)

# 2004년~2026년 필터링 후 저장
df_kospi_final = df_kospi.loc['2004-01-01':].sort_index()
df_kospi_final.to_csv('전처리/전처리_코스피.csv', encoding='utf-8-sig')

In [ ]:
# 환율 전처리
df_exch = df_exch_raw[['날짜', '종가']].copy()
df_exch.columns = ['date', 'exchange']
df_exch['date'] = pd.to_datetime(df_exch['date'].astype(str).str.replace(' ', ''))
df_exch = df_exch.set_index('date')
df_exch['exchange'] = df_exch['exchange'].astype(str).str.replace(',', '').astype(float)

# 2004년~2026년 필터링 후 저장
df_exch_final = df_exch.loc['2004-01-01':].sort_index()
df_exch_final.to_csv('전처리/전처리_환율.csv', encoding='utf-8-sig')

In [ ]:
# 파일 불러오기
filename = 'new_data/예금은행 대출금리(신규취급액 기준)_19101715.csv'
df = pd.read_csv(filename)
#  불필요한 메타데이터 컬럼 삭제 ('통계표', '단위', '변환' 등 제거)
df_clean = df.drop(columns=['통계표', '단위', '변환'])
# 행/열 뒤집기 (Transpose)
# 이제 날짜가 인덱스(행)가 되고, 대출 항목들이 컬럼(열)이 됩니다.
df_clean = df_clean.set_index('계정항목').T
# 날짜 인덱스 정리 (YYYY/MM -> datetime 형식)
df_clean.index = pd.to_datetime(df_clean.index, format='%Y/%m')
df_clean.index.name = 'date'
# 컬럼명 깔끔하게 정리 (공백 및 특수문자 제거)
# 예: "    가계대출 1)" -> "가계대출"
df_clean.columns = df_clean.columns.str.replace(r'\s*\d+\)', '', regex=True).str.strip()
# 숫자형으로 변환 (혹시 모를 문자열 오류 방지)
df_clean = df_clean.apply(pd.to_numeric, errors='coerce')
# 결과 확인
print(df_clean.head())
print(df_clean.info())
# 저장
df_clean.to_csv('전처리/전처리_대출금리.csv', encoding='utf-8-sig')